CASHBACK

In [2]:
#Library

import pandas as pd
import glob as glob
import numpy as np
import os as os
from datetime import datetime

pd.set_option('display.max_rows',None)

In [3]:
#CF

path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\1. CF"
file = glob.glob(path + '/*.csv')

CF = []

for fs in file:
    CF.append(pd.read_csv(fs, encoding = 'utf-8', header = 0))

CF = pd.concat(CF, ignore_index = True)
CF.rename(columns = {'结算网点' : 'Mitra', '费用金额' : 'Income', '客户编号' : 'Kode Pelanggan'}, inplace = True)

Income = CF.groupby('Mitra')['Income'].sum().reset_index()

C:\Users\JT-Cargo\AppData\Local\Temp\ipykernel_33572\1978761950.py:9: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  CF.append(pd.read_csv(fs, encoding = 'utf-8', header = 0))


In [4]:
#CF M-1 

path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\2. CF M-1"
file = glob.glob(path + '/*.csv')

CF1 = pd.concat([pd.read_csv(f, encoding = 'utf-8', header = 0) for f in file], ignore_index = True)
CF1.rename(columns = {'结算网点' : 'Mitra', '费用金额' : 'Income1','客户编号' : 'Kode Pelanggan'}, inplace = True)

Income1 = CF1.groupby('Mitra')['Income1'].sum().reset_index()

C:\Users\JT-Cargo\AppData\Local\Temp\ipykernel_33572\3543514826.py:6: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  CF1 = pd.concat([pd.read_csv(f, encoding = 'utf-8', header = 0) for f in file], ignore_index = True)
C:\Users\JT-Cargo\AppData\Local\Temp\ipykernel_33572\3543514826.py:6: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  CF1 = pd.concat([pd.read_csv(f, encoding = 'utf-8', header = 0) for f in file], ignore_index = True)


In [5]:
#Tonase

path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\3. Tonase"
file = glob.glob(path + '/*.csv')

Tonase = pd.concat([pd.read_csv(f, encoding = 'utf-8', header = 0) for f in file],ignore_index = True)
Tonase.rename(columns = {'寄件网点' : 'Mitra', '包裹计费重量' : 'Tonase', '客户编号' : 'Kode Pelanggan'}, inplace = True)

Ton = Tonase.groupby('Mitra')['Tonase'].sum().reset_index()

In [6]:
#VIP Customer

path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\4. VIP Customer"
file = glob.glob(path + '/*.csv')

VIP_Customer = pd.concat([pd.read_csv(f, encoding = 'utf-8', header = 0) for f in file], ignore_index = True)

In [7]:
#Mitra Upgrade

path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\5. Mitra Upgrade"
file = glob.glob(path + '/*.csv')

Mitra_Upgrade = pd.concat([pd.read_csv(f, header = 0, encoding = 'utf-8') for f in file], ignore_index = True)

In [8]:
#WorkDays

WD_Apr = 21
WD_Mei = 25.1
WD_Jun = 24.3
WD_Jul = 27.4
WD_Aug = 26.7
WD_Sep = 25.9
WD_Oct = 27.4
WD_Nov = 25.5
WD_Dec = 25.4

In [9]:
##Jakarta

path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\Jakarta"
file = glob.glob(path + '/*.csv')

Jakarta = pd.concat([pd.read_csv(f, encoding = 'utf-8', header = 0) for f in file], ignore_index = True)

Jakarta = (
    Jakarta
    .merge(Income, on = 'Mitra', how = 'left')
    .merge(Income1, on = 'Mitra', how = 'left')
)

Jakarta['Tipe Area'] = Jakarta['Tipe Area'].str.strip().astype('str')

Jakarta['lvl 2'] = Jakarta['Target Cashback'] * 1.3
Jakarta['lvl 3'] = Jakarta['Target Cashback'] * 1.5
Jakarta['Persentase Pencapaian'] = (Jakarta['Income']/Jakarta['Target Denda']).fillna(0)

pair = pd.concat([
    Mitra_Upgrade.set_index('Mitra')['Mitra Upgrade'],
    Mitra_Upgrade.set_index('Mitra Upgrade')['Mitra']]).to_dict() 

Income_pair = Income.set_index('Mitra')['Income']
Income_pair1 = Income1.set_index('Mitra')['Income1']

Jakarta['Mitra'] = Jakarta['Mitra'].str.strip()
Income['Mitra'] = Income['Mitra'].str.strip()

Jakarta['Income'] = Jakarta['Income'].fillna(0) + Jakarta['Mitra'].map(pair).map(Income_pair).fillna(0)
Jakarta['Income1'] = Jakarta['Income1'].fillna(0) + Jakarta['Mitra'].map(pair).map(Income_pair1).fillna(0)

#Cashback
a = Jakarta['lvl 3']      
b = Jakarta['lvl 2']     
c = Jakarta['Target Cashback']  
t = Jakarta['Tipe Area']
i = Jakarta['Income']
f = (Jakarta['Income'] - Jakarta['Income1']).clip(lower=0)

conditions = []
choices = []

rates = {
    'Basah': [(a, 0.20), (b, 0.17), (c, 0.14)],
    'Sedang': [(a, 0.16), (b, 0.13), (c, 0.10)],
    'Kering': [(a, 0.13), (b, 0.10), (c, 0.08)],
}

for area, rules in rates.items():
    for target, rate in rules:
        conditions.append((Jakarta['Income'] > target) & (Jakarta['Tipe Area'] == area))
        choices.append(f * rate)

cb_raw = np.select(conditions, choices, default=0)

Jakarta['Cashback'] = np.where(
    cb_raw > 0,
    (cb_raw - Jakarta['Pengembalian Denda']).clip(lower = 0),
    0
)

#Denda
p = Jakarta['Persentase Pencapaian']

conditions = []
choices = []

rates = {
    'Basah' : [(0.7,1000000), (0.8,800000), (0.9,500000)],
    'Sedang' : [(0.7, 800000), (0.8, 600000), (0.9, 400000)],
    'Kering' : [(0.7, 600000), (0.8, 400000), (0.9, 200000)]
}

for area, rules in rates.items():
    for target, rate in rules:
        conditions.append((p < target) & (t == area))
        choices.append(rate * -1)

Jakarta['Denda'] = np.select(conditions, choices, default = 0)

Jakarta['Cashback Check'] = np.where((Jakarta['Cashback'] == Jakarta['Cashback Area']) | (abs(Jakarta['Cashback'] - Jakarta['Cashback Area']) < 1), 'TRUE','FALSE')
Jakarta['Selisih Cashback'] = Jakarta['Cashback'] - Jakarta['Cashback Area']
Jakarta['Denda Check'] = np.where(Jakarta['Denda'] == Jakarta['Denda Area'], 'TRUE','FALSE')
Jakarta['Selisih Denda'] = Jakarta['Denda'] - Jakarta['Denda Area']

'''
NonDenda = ['JKT053A', 'JKT251B', 'JKT062B']
Jakarta['Denda'] = np.where(Jakarta.Mitra.isin(NonDenda),0, Jakarta['Denda'])
'''

path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\0. Data\Oct 25"
Jakarta.to_csv(path + '\Jakarta Cashback_Oct 25.csv', index = False)

<>:93: SyntaxWarning: invalid escape sequence '\J'
<>:93: SyntaxWarning: invalid escape sequence '\J'
C:\Users\JT-Cargo\AppData\Local\Temp\ipykernel_33572\1698155485.py:93: SyntaxWarning: invalid escape sequence '\J'
  Jakarta.to_csv(path + '\Jakarta Cashback_Oct 25.csv', index = False)


In [10]:
##Tangerang

Path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\Tangerang"
file = glob.glob(Path + '/*.csv')

Tangerang = pd.concat([pd.read_csv(f, encoding = 'utf-8', header = 0) for f in file], ignore_index = True)

Tangerang = (
    Tangerang
    .merge(Income, on = 'Mitra', how = 'left')
    .merge(Income1, on = 'Mitra', how = 'left')
)

pair = pd.concat([
    Mitra_Upgrade.set_index('Mitra')['Mitra Upgrade'],
    Mitra_Upgrade.set_index('Mitra Upgrade')['Mitra']]).to_dict()

Income_pair = Income.set_index('Mitra')['Income']
Income1_pair = Income1.set_index('Mitra')['Income1']

Tangerang['Income'] = Tangerang['Income'].fillna(0) + Tangerang['Mitra'].map(pair).map(Income_pair).fillna(0)
Tangerang['Income1'] = Tangerang['Income1'].fillna(0) + Tangerang['Mitra'].map(pair).map(Income1_pair).fillna(0)

Tangerang['Persentase Pencapaian'] = ((Tangerang['Income']/WD_Oct)/(Tangerang['Income1']/WD_Sep)).astype(float)

#Cashback
i = Tangerang['Income']
t = Tangerang['Target Cashback']

Tangerang['Cashback'] = (np.where(i > t, (i - t)*0.5,0) - Tangerang['PIC Reward']).clip(lower = 0)

#Denda
p = Tangerang['Persentase Pencapaian']
ta = Tangerang['Tipe Area']

basah = 'Area Basah'
sedang = 'Area Sedang'
kering = 'Area Kering'


conditions = []
choices = []

rates = {
    basah : [(0.7, 1000000), (0.8, 800000), (0.9, 500000)],
    sedang : [(0.7, 800000), (0.8, 600000), (0.9, 400000)],
}

for area, rule in rates.items():
    for target, rate in rule:
        conditions.append((p < target) & (ta == area))
        choices.append(rate * -1)

Tangerang['Denda'] = np.where(t > 30000000, np.select(conditions,choices,0),0)

Tangerang['Cashback Check'] = np.where((Tangerang['Cashback'] == Tangerang['Cashback Area']) | (abs(Tangerang['Cashback'] - Tangerang['Cashback Area'])<1),'TRUE','FALSE')
Tangerang['Selisih Cashback'] = Tangerang['Cashback'] - Tangerang['Cashback Area']
Tangerang['Denda Check'] = np.where((Tangerang['Denda'] == Tangerang['Denda Area']) | (abs(Tangerang['Denda'] - Tangerang['Denda Area'])<1),'TRUE','FALSE')
Tangerang['Selisih Denda'] = Tangerang['Denda'] - Tangerang['Denda Area']

path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\0. Data\Oct 25"
Tangerang.to_csv(path + '\Tangerang Cashback_Oct 25.csv', index = False)

<>:62: SyntaxWarning: invalid escape sequence '\T'
<>:62: SyntaxWarning: invalid escape sequence '\T'
C:\Users\JT-Cargo\AppData\Local\Temp\ipykernel_19444\3519537884.py:62: SyntaxWarning: invalid escape sequence '\T'
  Tangerang.to_csv(path + '\Tangerang Cashback_Oct 25.csv', index = False)


In [11]:
#Bandung

Path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\Bandung"
File = glob.glob(Path + '/*.csv')

Bandung = pd.concat([pd.read_csv(f, encoding = 'utf-8', header = 0) for f in File], ignore_index = True)

Tonase = Tonase.loc[~Tonase['Kode Pelanggan'].isin(VIP_Customer['Kode Pelanggan'])]
Tonase = Tonase.groupby('Mitra')['Tonase'].sum().reset_index()

Income = CF.loc[~CF['Kode Pelanggan'].isin(VIP_Customer['Kode Pelanggan'])]
Income = Income.groupby('Mitra')['Income'].sum().reset_index()

Bandung = (
    Bandung
    .merge(Income, on = 'Mitra', how = 'left')
    .merge(Tonase, on = 'Mitra', how = 'left')
).fillna(0)

pair = pd.concat([
    Mitra_Upgrade.set_index('Mitra')['Mitra Upgrade'],
    Mitra_Upgrade.set_index('Mitra Upgrade')['Mitra']]).to_dict()

Tonase_pair = Tonase.set_index('Mitra')['Tonase']
Income_pair = Income.set_index('Mitra')['Income']

Bandung['Income'] = Bandung.Income.fillna(0) + Bandung.Mitra.map(pair).map(Income_pair).fillna(0)
Bandung['Tonase'] = Bandung.Tonase.fillna(0) + Bandung.Mitra.map(pair).map(Tonase_pair).fillna(0)
Bandung['Tipe Mitra'] = Bandung['Mitra'].str[-1]
Bandung['Persentase Pencapaian'] = np.where(Bandung['Target Tonase'] == 0, 1, Bandung['Tonase']/Bandung['Target Tonase'])

#Cashback
pc = Bandung['Partisipasi Cashback']
i = Bandung['Income']
tc = Bandung['Target Cashback']

Bandung['Cashback'] = np.where((pc == 'Ikut Program') & (i > tc), (i - tc) * 0.2,0)

#Denda
pp = Bandung['Persentase Pencapaian']
ta = Bandung['Tipe Area']
tm = Bandung['Tipe Mitra']
basah = 'Area Basah'
sedang = 'Area Sedang'
kering = 'Area Kering'

conditions = []
choices = []

rates = {
    basah : [("A", 0.7, 2000000),("A", 0.8, 1000000),("A", 0.9, 800000),("B", 0.7,  1000000),("B", 0.8, 500000),("B", 0.9, 400000)],
    sedang : [("A", 0.7, 1000000),("A", 0.8, 800000),("A", 0.9, 600000),("B", 0.7,  500000),("B", 0.8, 400000),("B", 0.9, 300000)]
}

for area, rule in rates.items():
    for tipe, target, rate in rule:
        conditions.append((ta == area) & (tm == tipe) & (pp < target))
        choices.append(rate * -1)

Bandung['Denda'] = np.select(conditions, choices, 0)

Bandung['Cashback Check'] = np.where((Bandung['Cashback Area'] == Bandung['Cashback']) | (abs(Bandung['Cashback Area'] - Bandung['Cashback']) < 1), 'TRUE','FALSE')
Bandung['Selisih Cashback'] = Bandung['Cashback Area'] - Bandung['Cashback']

Bandung['Denda Check'] = np.where(Bandung['Denda Area'] == Bandung['Denda'], 'TRUE', 'FALSE')
Bandung['Selisih Denda'] = Bandung['Denda Area'] - Bandung['Denda']

path = r"C:\Users\JT-Cargo\Desktop\Pricing Strategy\0. Python\Cashback\0. Data\Oct 25"
Bandung.to_csv(path + '\Bandung Cashback_Oct 25.csv', index = False) 

<>:69: SyntaxWarning: invalid escape sequence '\B'
<>:69: SyntaxWarning: invalid escape sequence '\B'
C:\Users\JT-Cargo\AppData\Local\Temp\ipykernel_19444\938819197.py:69: SyntaxWarning: invalid escape sequence '\B'
  Bandung.to_csv(path + '\Bandung Cashback_Oct 25.csv', index = False)
